In [10]:
import pandas as pd
import numpy as np
import requests
import os

# DATA DIRECTORY CONFIGURATION 
DATA_DIR = '../data/raw'
CACHE_FILE = f'{DATA_DIR}/sp500_tickers.csv'

# Ensure the directory exists
os.makedirs(DATA_DIR, exist_ok=True)

def get_sp500_tickers():
    
    #Fetches S&P 500 tickers from Wikipedia
    #If the file exists locally, it loads it. If not, it scrapes.
    
    # 1. CHECK CACHE
    if os.path.exists(CACHE_FILE):
        print(f"✅ Loading tickers from cache: {CACHE_FILE}")
        return pd.read_csv(CACHE_FILE)

    # 2. SCRAPE WIKIPEDIA (If cache missing)
    print("⚠️ Cache not found. Scraping Wikipedia...")
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        r = requests.get(url, headers=headers)
        tables = pd.read_html(r.text)
        
        # Robust Table Search: Find the one with "GICS Sector"
        sp500_table = next((t for t in tables if "GICS Sector" in t.columns), None)
        
        if sp500_table is None:
            raise ValueError("Could not find S&P 500 table.")

        # 3. CLEAN SYMBOLS
        # Yahoo Finance uses '-' instead of '.' (e.g., BRK.B -> BRK-B)
        sp500_table['Symbol'] = sp500_table['Symbol'].str.replace('.', '-', regex=False)
        
        # 4. SAVE TO CACHE
        sp500_table.to_csv(CACHE_FILE, index=False)
        print(f"✅ Scraped and saved {len(sp500_table)} tickers to {CACHE_FILE}")
        return sp500_table

    except Exception as e:
        print(f"❌ Error scraping: {e}")
        # Fallback list just in case
        return pd.DataFrame({
            'Symbol': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA'], 
            'GICS Sector': ['Information Technology']*5
        })

# EXECUTION 
df_tickers = get_sp500_tickers()

# group tickers by each corresponding GICS sector in order to creat a dataframe for each sector
tickers_by_sector = df_tickers.groupby('GICS Sector')['Symbol'].apply(list).to_dict()

print(f"\nReady to process {len(tickers_by_sector)} sectors.")
print(f"Sample: {list(tickers_by_sector.keys())[:3]}")

✅ Loading tickers from cache: ../data/raw/sp500_tickers.csv

Ready to process 11 sectors.
Sample: ['Communication Services', 'Consumer Discretionary', 'Consumer Staples']


In [11]:

#DATA DOWNLOADER AND PROCESSOR
#turn tickers list into price and returns dataframes for each sector

import yfinance as yf
import pandas as pd
import os

print(f"Starting download for {len(tickers_by_sector)} sectors...")

for sector_name, sector_tickers in tickers_by_sector.items():
    # Clean sector name for filename (remove spaces/special chars)
    safe_sector_name = sector_name.replace(" ", "_").lower()
    print(f"\n Processing Sector: {sector_name} ")
    
    # Download both Closing price and Volume data
    # We create a list of tickers to ensure yfinance treats it as a batch
    try:
        raw_data = yf.download(
            list(sector_tickers), 
            start="2021-01-01", 
            end="2025-12-31", 
            auto_adjust=True, 
            threads=True,
            group_by='column' # Helps structure the multi-index
        )
    except Exception as e:
        print(f"Failed to download {sector_name}: {e}")
        continue

    # Check if data is empty
    if raw_data.empty:
        print(f"No data found for {sector_name}")
        continue

    # Extract Closing price and Volume frames
    try:
        df_close = raw_data['Close']
        df_volume = raw_data['Volume']
    except KeyError:
        print(f"Data structure error for {sector_name}, skipping.")
        continue


    print(f"  Initial count: {df_close.shape[1]} stocks")
    
    # Filter by Volume - LIQUIDITY CHECK
    # Calculate average daily volume for each stock
    avg_volume = df_volume.mean()
    
    # Define the volume Threshold: Drop the bottom 25% (lowest liquidity stocks)
    volume_threshold = avg_volume.quantile(0.25) 
    
    # Identify liquid tickers
    liquid_tickers = avg_volume[avg_volume >= volume_threshold].index
    
    print(f"  Volume Threshold: {volume_threshold:.0f} avg shares/day")
    print(f"  Dropping {len(sector_tickers) - len(liquid_tickers)} low-volume stocks...")
    
    # Keep only liquid stocks in the price dataframe
    df_close = df_close[liquid_tickers]

    # 3. Handle Missing Data
    # First, drop columns that are entirely NaN
    df_close = df_close.dropna(axis=1, how='all')
    
    # Calculate returns to check for data validity
    sector_returns = df_close.pct_change()
    
    # Filter: Keep stocks with at least 90% valid data points
    min_data_points = 0.9 * len(sector_returns)
    sector_returns = sector_returns.dropna(thresh=min_data_points, axis=1)
    
    # Forward fill prices/returns to handle small gaps, then drop remaining NaNs
    sector_returns = sector_returns.ffill().dropna()
    
    # Re-align prices to match the final cleaned returns columns
    final_prices = df_close[sector_returns.columns].loc[sector_returns.index]

    print(f"  Final Shape: {final_prices.shape}")

    # 4. Save Data
    # Save Prices
    price_filename = f'../data/raw/{safe_sector_name}_prices.csv'
    final_prices.to_csv(price_filename)
    
    # Save Returns (for PCA/Clustering)
    returns_filename = f'../data/processed/{safe_sector_name}_returns.csv'
    sector_returns.to_csv(returns_filename)
    
    print(f"  Saved -> {safe_sector_name}")

print("\nAll sectors processed.")

Starting download for 11 sectors...

 Processing Sector: Communication Services 


[*********************100%***********************]  23 of 23 completed


  Initial count: 23 stocks
  Volume Threshold: 2165402 avg shares/day
  Dropping 6 low-volume stocks...
  Final Shape: (1253, 17)
  Saved -> communication_services

 Processing Sector: Consumer Discretionary 


[*********************100%***********************]  49 of 49 completed


  Initial count: 49 stocks
  Volume Threshold: 1784052 avg shares/day
  Dropping 12 low-volume stocks...
  Final Shape: (1253, 37)
  Saved -> consumer_discretionary

 Processing Sector: Consumer Staples 


...

  Initial count: 26 stocks
  Volume Threshold: 1405787 avg shares/day
  Dropping 7 low-volume stocks...
  Final Shape: (1253, 18)
  Saved -> materials

 Processing Sector: Real Estate 


[*********************100%***********************]  31 of 31 completed


  Initial count: 31 stocks
  Volume Threshold: 963515 avg shares/day
  Dropping 8 low-volume stocks...
  Final Shape: (1253, 23)
  Saved -> real_estate

 Processing Sector: Utilities 


[*********************100%***********************]  31 of 31 completed


  Initial count: 31 stocks
  Volume Threshold: 1927416 avg shares/day
  Dropping 8 low-volume stocks...
  Final Shape: (1253, 22)
  Saved -> utilities

All sectors processed.
